# మోడల్ కాంటెక్స్ట్ ప్రోటోకాల్ (MCP) మద్దతుతో Azure AI ఏజెంట్లు

ఈ నోట్‌బుక్ Azure AI ఏజెంట్లను మోడల్ కాంటెక్స్ట్ ప్రోటోకాల్ (MCP) టూల్స్‌తో ఎలా ఉపయోగించాలో చూపిస్తుంది, ఇది మెరుగైన సామర్థ్యాల కోసం బాహ్య MCP సర్వర్లను ఉపయోగించగల తెలివైన ఏజెంట్‌ను సృష్టిస్తుంది.


## Install Required NuGet Packages

మొదట, Azure AI Agents తో పని చేయడానికి ప్రాథమిక ఫంక్షనాలిటీని అందించే Azure AI Agents Persistent ప్యాకేజీని ఇన్‌స్టాల్ చేయాలి.


## కీ లేని ప్రమాణీకరణ లాభాలు

ఈ నోట్‌బుక్ **కీ లేని ప్రమాణీకరణ**ని ప్రదర్శిస్తుంది, ఇది అనేక ప్రయోజనాలను అందిస్తుంది:
- ✅ **నిర్వహించాల్సిన ఏపీఐ కీలు లేవు** - Azure ఐడెంటిటీ ఆధారిత ప్రమాణీకరణను ఉపయోగిస్తుంది
- ✅ **పెరిగిన భద్రత** - కోడ్ లేదా కాన్ఫిగరేషన్‌లో రహస్యాలు నిల్వ చేయబడవు
- ✅ **స్వయంచాలక క్రెడెన్షియల్ రొటేషన్** - క్రెడెన్షియల్ జీవనచక్రాన్ని Azure నిర్వహిస్తుంది
- ✅ **పాత్ర ఆధారిత యాక్సెస్** - సన్నిహిత అనుమతుల కోసం Azure RBAC ఉపయోగిస్తుంది

`DefaultAzureCredential` ఆటోమేటిక్‌గా ఉత్తమంగా అందుబాటులో ఉన్న క్రెడెన్షియల్ మూలాన్ని ఉపయోగిస్తుంది:
1. మేనేజ్డ్ ఐడెంటిటీ (Azureలో నడుస్తున్నప్పుడు)
2. Azure CLI క్రెడెన్షియల్స్ (డెవలప్‌మెంట్ సమయంలో)
3. Visual Studio క్రెడెన్షియల్స్
4. పర్యావరణ వేరియబుల్స్ (కాన్ఫిగర్ చేసినట్లయితే)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

Azure సేవలతో DefaultAzureCredential ఉపయోగించి ప్రామాణీకరణ కోసం Azure Identity ప్యాకేజీని ఇన్‌స్టాల్ చేయండి.


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## అవసరమైన నేమ్‌స్పేస్‌లను దిగుమతి చేసుకోండి

Azure AI ఏజెంట్లు మరియు Azure ఐడెంటిటీ కోసం అవసరమైన నేమ్‌స్పేస్‌లను దిగుమతి చేసుకోండి.


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Azure AI ఏజెంట్ క్లయింట్‌ను కాన్ఫిగర్ చేయండి (కీలెస్ ఆథెంటికేషన్)

కాన్ఫిగరేషన్ వేరియబుల్స్‌ను సెట్ చేసి **కీలెస్ ఆథెంటికేషన్** ఉపయోగించి PersistentAgentsClient ను సృష్టించండి:
- **projectEndpoint**: Azure AI Foundry ప్రాజెక్ట్ ఎండ్‌పాయింట్
- **modelDeploymentName**: డిప్లాయ్ చేసిన AI మోడల్ పేరు (GPT-4.1 నానో)
- **mcpServerUrl**: MCP సర్వర్ URL (Microsoft Learn API)
- **mcpServerLabel**: MCP సర్వర్‌ను గుర్తించడానికి లేబుల్
- **DefaultAzureCredential**: మేనేజ్డ్ ఐడెంటిటీ, Azure CLI లేదా ఇతర క్రెడెన్షియల్ మూలాలను ఉపయోగిస్తుంది (ఏ API కీలు అవసరం లేదు)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## MCP టూల్ నిర్వచనం సృష్టించండి

Microsoft Learn MCP సర్వర్‌కు కనెక్ట్ అయ్యే MCP టూల్ నిర్వచనాన్ని సృష్టించండి. ఇది ఏజెంట్‌కు Microsoft Learn కంటెంట్ మరియు డాక్యుమెంటేషన్‌ను యాక్సెస్ చేయడానికి అనుమతిస్తుంది.


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## AI ఏజెంట్ సృష్టించండి

నిర్దిష్ట మోడల్ మరియు MCP టూల్స్‌తో ఒక స్థిరమైన AI ఏజెంట్‌ను సృష్టించండి. ఏజెంట్ ఈ విధంగా కాన్ఫిగర్ చేయబడింది:
- GPT-4.1 నానో మోడల్
- సహాయానికి MCP టూల్స్ ఉపయోగించడానికి సూచనలు
- Microsoft Learn MCP సర్వర్‌కు ప్రాప్యత


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## థ్రెడ్ సృష్టించండి మరియు సందేశం పంపండి

ఒక సంభాషణ థ్రెడ్ సృష్టించి, Azure OpenAI మరియు OpenAI మధ్య తేడా గురించి అడుగుతూ ఒక వినియోగదారుని సందేశం పంపండి. ఇది ఏజెంట్ MCP టూల్స్ ఉపయోగించి సరిగ్గా సమాచారం అందించే సామర్థ్యాన్ని పరీక్షిస్తుంది.


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## MCP టూల్ వనరులను కాన్ఫిగర్ చేయండి (కీలెస్)

MCP టూల్ వనరులను సెటప్ చేయండి. నిజంగా కీలెస్ విధానానికి, MCP సర్వర్ Azure ఐడెంటిటీ ఆధారిత ప్రమాణీకరణను మద్దతు ఇస్తే, మీరు కస్టమ్ హెడ్డర్లను తీసివేయవచ్చు. అవసరమైతే హెడ్డర్లను ఎలా జోడించాలో క్రింది ఉదాహరణ చూపిస్తుంది, కానీ కీలెస్ సందర్భాల్లో ఇవి అవసరం ఉండకపోవచ్చు.


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## ఏజెంట్ రన్ ప్రారంభించండి

వినియోగదారుడి సందేశాన్ని ప్రాసెస్ చేయడానికి ఒక రన్ సృష్టించి ప్రారంభించండి. ఏజెంట్ ప్రతిస్పందనను ఉత్పత్తి చేయడానికి కాన్ఫిగర్ చేసిన MCP టూల్స్ మరియు వనరులను ఉపయోగిస్తుంది.


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## రన్‌ను పర్యవేక్షించండి మరియు టూల్ ఆమోదాలను నిర్వహించండి (కీలెస్)

ఏజెంట్ రన్ స్థితిని పర్యవేక్షించండి మరియు అవసరమైన టూల్ ఆమోదాలను నిర్వహించండి. ఈ లూప్:
1. రన్ పూర్తవ్వడానికి లేదా చర్య అవసరమయ్యే వరకు వేచి ఉంటుంది
2. అవసరమైనప్పుడు MCP టూల్ కాల్స్‌ను ఆటోమేటిక్‌గా ఆమోదిస్తుంది
3. కీలెస్ ఆథెంటికేషన్ కోసం, MCP సర్వర్ Azure ఐడెంటిటీని మద్దతు ఇస్తే హెడ్డర్లు అవసరం ఉండకపోవచ్చు


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## సంభాషణ ఫలితాలను ప్రదర్శించండి

థ్రెడ్‌లోని అన్ని సందేశాలను పొందండి మరియు ప్రదర్శించండి, వినియోగదారుడి ప్రశ్న మరియు ఏజెంట్ యొక్క ప్రతిస్పందన రెండింటినీ చూపిస్తూ. సందేశాలు కాలానుక్రమంలో టైమ్‌స్టాంప్‌లు మరియు పాత్ర సూచికలతో ప్రదర్శించబడతాయి.


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**అస్పష్టత**:  
ఈ డాక్యుమెంట్‌ను AI అనువాద సేవ [Co-op Translator](https://github.com/Azure/co-op-translator) ఉపయోగించి అనువదించబడింది. మేము ఖచ్చితత్వానికి ప్రయత్నించినప్పటికీ, ఆటోమేటెడ్ అనువాదాల్లో పొరపాట్లు లేదా తప్పిదాలు ఉండవచ్చు. అసలు డాక్యుమెంట్ దాని స్వదేశీ భాషలోనే అధికారిక మూలంగా పరిగణించాలి. ముఖ్యమైన సమాచారానికి, ప్రొఫెషనల్ మానవ అనువాదం సిఫార్సు చేయబడుతుంది. ఈ అనువాదం వాడకంలో ఏర్పడిన ఏవైనా అపార్థాలు లేదా తప్పుదారితీసే అర్థాలు కోసం మేము బాధ్యత వహించము.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
